In [2]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier

from scipy import interp

from itertools import cycle

import seaborn as sns

In [6]:
data_dir = './sounds'
audio_files = glob(data_dir + '/*')

In [8]:
from pydub import AudioSegment
from pydub.utils import make_chunks

for j in range(len(audio_files)):
    myaudio = AudioSegment.from_file("./sounds/{}".format(os.listdir('./sounds/')[j])) 
    chunk_length_ms = 10000 # pydub calculates in millisec
    chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

    #Export all of the individual chunks as wav files

    for i, chunk in enumerate(chunks):
        chunk_name = "{}{}.wav".format(os.listdir('./sounds/')[j], i)
        print("exporting", chunk_name)
        chunk.export(chunk_name, format="mp3")

exporting vacuum.mp30.wav
exporting vacuum.mp31.wav
exporting vacuum.mp32.wav
exporting vacuum.mp33.wav
exporting vacuum.mp34.wav
exporting vacuum.mp35.wav
exporting vacuum.mp36.wav
exporting vacuum.mp37.wav
exporting vacuum.mp38.wav
exporting vacuum.mp39.wav
exporting vacuum.mp310.wav
exporting vacuum.mp311.wav
exporting vacuum.mp312.wav
exporting vacuum.mp313.wav
exporting vacuum.mp314.wav
exporting vacuum.mp315.wav
exporting vacuum.mp316.wav
exporting vacuum.mp317.wav
exporting vacuum.mp318.wav
exporting vacuum.mp319.wav
exporting vacuum.mp320.wav
exporting vacuum.mp321.wav
exporting vacuum.mp322.wav
exporting vacuum.mp323.wav
exporting vacuum.mp324.wav
exporting vacuum.mp325.wav
exporting vacuum.mp326.wav
exporting vacuum.mp327.wav
exporting vacuum.mp328.wav
exporting vacuum.mp329.wav
exporting vacuum.mp330.wav
exporting vacuum.mp331.wav
exporting vacuum.mp332.wav
exporting vacuum.mp333.wav
exporting vacuum.mp334.wav
exporting vacuum.mp335.wav
exporting vacuum.mp336.wav
exporting v

exporting washing_machine_2.mp317.wav
exporting washing_machine_2.mp318.wav
exporting washing_machine_2.mp319.wav
exporting washing_machine_2.mp320.wav
exporting washing_machine_2.mp321.wav
exporting washing_machine_2.mp322.wav
exporting washing_machine_2.mp323.wav
exporting washing_machine_2.mp324.wav
exporting washing_machine_2.mp325.wav
exporting washing_machine_2.mp326.wav
exporting washing_machine_2.mp327.wav
exporting washing_machine_2.mp328.wav
exporting washing_machine_2.mp329.wav
exporting washing_machine_2.mp330.wav
exporting washing_machine_2.mp331.wav
exporting washing_machine_2.mp332.wav
exporting washing_machine_2.mp333.wav
exporting washing_machine_2.mp334.wav
exporting washing_machine_2.mp335.wav
exporting washing_machine_2.mp336.wav
exporting washing_machine_2.mp337.wav
exporting washing_machine_2.mp338.wav
exporting washing_machine_2.mp339.wav
exporting washing_machine_2.mp340.wav
exporting washing_machine_2.mp341.wav
exporting washing_machine_2.mp342.wav
exporting wa

In [7]:
len(audio_files)

8

In [9]:
data_dir = './wav2'
audio_files = glob(data_dir + '/*.wav')

In [10]:
len(audio_files)

489

In [11]:
data = pd.DataFrame(columns = ['name','mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse'])

In [12]:
lista = []
for i in range(len(audio_files)):
    lista.append(os.listdir('./wav2/')[i])
data.name = lista

In [13]:
lista2 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    mfcc_aux = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    mfcc = np.ndarray.mean(mfcc_aux)
    lista2.append(mfcc)
data.mfcc = lista2

In [14]:
lista3 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    scem = np.ndarray.mean(cent)
    lista3.append(scem)
data.scem = lista3

In [15]:
lista4 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    S = np.abs(librosa.stft(y))
    contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
    scom = np.ndarray.mean(contrast)
    lista4.append(scom)
data.scom = lista4

In [16]:
lista5 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    srom = np.ndarray.mean(rolloff)
    lista5.append(srom)
data.srom = lista5

In [17]:
lista6 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    sbwm = np.ndarray.mean(spec_bw)
    lista6.append(sbwm)
data.sbwm = lista6

In [18]:
lista7 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    hop_length = 512
    oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
    tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=hop_length)

    # Compute global onset autocorrelation
    ac_global = librosa.autocorrelate(oenv, max_size=tempogram.shape[0])
    ac_global = librosa.util.normalize(ac_global)

    # Estimate the global tempo for display purposes
    tempo = librosa.beat.tempo(onset_envelope=oenv, sr=sr, hop_length=hop_length)[0]
    lista7.append(tempo)
data.tempo = lista7

In [19]:
lista8 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    rmse_aux = librosa.feature.rms(y=y)
    rmse = np.ndarray.mean(rmse_aux)
    lista8.append(rmse)
data.rmse = lista8

In [20]:
data.shape

(489, 8)

In [21]:
data.dtypes

name      object
mfcc     float64
scem     float64
scom     float64
srom     float64
sbwm     float64
tempo    float64
rmse     float64
dtype: object

In [22]:
momento = []

for i in data['name']:
    if 'ducha' in i:
        momento.append('Ducha')
    elif 'cena' in i:
        momento.append('Cena')
    elif 'washing' in i:
        momento.append('Lavadora')
    elif 'vacuum' in i:
        momento.append('Aspiradora')
    else:
        momento.append('Otro')
        
data['momento_aux'] = momento
#data['momento'] = 'Otro'
data.drop(['name'], axis=1, inplace=True)

In [25]:
data.momento_aux.value_counts()

Lavadora      244
Cena          130
Aspiradora     80
Ducha          35
Name: momento_aux, dtype: int64

In [26]:
data.to_csv('data_momento_aux.csv', index=False)